In [101]:
%matplotlib inline

import numpy as np
import pandas as pd

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from pydub import AudioSegment
from pydub.utils import make_chunks

import re

In [102]:
def decompose_files(data_dir, audio_files):
    for j in range(len(audio_files)):
        myaudio = AudioSegment.from_file(data_dir + '{}'.format(os.listdir(data_dir)[j])) 
        chunk_length_ms = 10000
        chunks = make_chunks(myaudio, chunk_length_ms)

        for i, chunk in enumerate(chunks):
            chunk_name = "{}{}.wav".format(os.listdir(data_dir)[j], i)
            print("exporting", chunk_name)
            chunk.export('./sounds2/{}'.format(chunk_name), format="mp3")

In [103]:
decompose_files('./sounds/', glob('./sounds' + '/*'))

exporting doorbell.mp30.wav
exporting doorbell.mp31.wav
exporting doorbell.mp32.wav
exporting doorbell.mp33.wav
exporting doorbell.mp34.wav
exporting doorbell.mp35.wav
exporting doorbell.mp36.wav
exporting doorbell.mp37.wav
exporting doorbell.mp38.wav
exporting doorbell.mp39.wav
exporting doorbell.mp310.wav
exporting doorbell.mp311.wav
exporting doorbell.mp312.wav
exporting doorbell.mp313.wav
exporting doorbell.mp314.wav
exporting doorbell.mp315.wav
exporting doorbell.mp316.wav
exporting doorbell.mp317.wav
exporting doorbell.mp318.wav
exporting doorbell.mp319.wav
exporting doorbell.mp320.wav
exporting doorbell.mp321.wav
exporting doorbell.mp322.wav
exporting doorbell.mp323.wav
exporting doorbell.mp324.wav
exporting doorbell.mp325.wav
exporting doorbell.mp326.wav
exporting doorbell.mp327.wav
exporting doorbell.mp328.wav
exporting doorbell.mp329.wav
exporting doorbell.mp330.wav
exporting doorbell.mp331.wav
exporting doorbell.mp332.wav
exporting doorbell.mp333.wav
exporting doorbell.mp334

In [104]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 'shaver': 'Afeitadora',
                'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 'cellphone': 'Telefono', 
                'comp_type': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 'blender': 'Licuadora', 
                'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i])
        name = os.listdir(data_dir)[i]
        pattern = "[.][\w]+"
        name = re.sub(pattern, '', name)
        if name in momentos.keys():
            momento = momentos.get(name)
        else:
            momento = 'Otro'
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        scem = np.ndarray.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        scom = np.ndarray.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(y)), sr=sr))
        srom = np.ndarray.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        sbwm = np.ndarray.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        tempo = librosa.beat.tempo(onset_envelope=librosa.onset.onset_strength(y=y, sr=sr, hop_length=512), sr=sr, hop_length=512)[0]
        rmse = np.ndarray.mean(librosa.feature.rms(y=y))
        features.append([mfcc, scem, scom, srom, sbwm, tempo, rmse, momento])
    return pd.DataFrame(features, columns=['mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 'momento'])

In [105]:
data = get_features('./sounds2/', glob('./sounds2' + '/*.wav'))
data.head()

,mfcc,scem,scom,srom,sbwm,tempo,rmse,momento
0,-8.979329,1772.598525,24.046327,3065.983394,1969.223814,198.768029,0.052725,Timbre
1,-8.775426,1783.777821,24.127860,3094.760946,1968.643861,198.768029,0.053559,Timbre
2,-15.435374,2476.533897,25.183671,3901.306804,2040.348226,117.453835,0.068441,Alarma
3,-8.908068,1766.532029,24.183965,3048.497034,1960.584538,198.768029,0.053536,Timbre
4,-15.547997,2469.838603,25.153774,3885.494138,2035.490056,117.453835,0.069116,Alarma


In [106]:
data.shape

(92, 8)

In [107]:
data.dtypes

mfcc       float64
scem       float64
scom       float64
srom       float64
sbwm       float64
tempo      float64
rmse       float64
momento     object
dtype: object

In [108]:
data.momento.value_counts()

Timbre    61
Alarma    31
Name: momento, dtype: int64

In [ ]:
#data.to_csv('data_momento_aux_2.csv', index=False)